In [19]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [22]:
student_data = {
    "bio": pd.read_csv("bio.csv"),
    "net": pd.read_csv("net.csv"),
    "gate": pd.read_csv("gate.csv"),
    "tenth": pd.read_csv("tenth.csv"),
    "others": pd.read_csv("others.csv"),
    "masters": pd.read_csv("masters.csv"),
    "plustwo": pd.read_csv("plustwo.csv"),
    "bachelors": pd.read_csv("bachelors.csv"),
    "experience": pd.read_csv("experience.csv"),
}

student_shortlisted_data = {}

for key in student_data.keys():
    student_shortlisted_data[key] = pd.DataFrame(columns = student_data[key].columns.tolist()) #copy column headers
    if "Institute" in student_data[key].keys():
        student_data[key]["Institute"].fillna("NA",inplace=True) #Replace all NaN values with "NA"
    if "Programme" in student_data[key].keys():
        student_data[key]["Programme"].fillna("NA",inplace=True) #Replace all NaN values with "NA"
    if "Specialization" in student_data[key].keys():
        student_data[key]["Specialization"].fillna("NA",inplace=True) #Replace all NaN values with "NA"
    if "Discipline" in student_data[key].keys():
        student_data[key]["Discipline"].fillna("NA",inplace=True) #Replace all NaN values with "NA"

student_data["net"] #verification

,Discipline,Year
0,NA,NaN
1,NA,NaN
2,COMPUTER SCIENCE AND APPLICATION,2020.0
3,NA,NaN
4,NA,NaN
5,COMPUTER SCIENCE AND APPLICATIONS,2020.0
6,COMPUTER SCIENCE AND APPLICATION,2019.0
7,NA,NaN
8,NA,NaN
9,NA,NaN


In [24]:

def checkIIT(Data):
    """
    This Function checks for IITians, just a query check that's all
    SHORTLISTING IIT CANDIDATES WITH COOL CG
    """
    mydf = pd.DataFrame()
    for i,row in Data.iterrows():
        if "iit" in row["Institute"].lower() or "indian institute of technology" in row["Institute"].lower():
            mydf = mydf.append(row,ignore_index=True)
    return mydf

def checkNET(Data):
    """
    This function Checks for NET-UGC Qualification
    """
    mydf = pd.DataFrame()
    for i,row in Data.iterrows():
        if row["Discipline"] != "NA":
            mydf = mydf.append(row)
    return mydf

#checkNET(student_data["net"]).index.tolist()

[2, 5, 6, 10, 33, 38, 41, 43, 45]